# Create perturbed FY files with ad-hoc covariance matrix (all energies)

This notebook follows the procedure to generate ad-hoc covariance matrices for independent fission yields from chain yield evaluations, as described in:
- [Fiorito et al., Generation of fission yield covariances to correct discrepancies in the nuclear data libraries, Annals of Nuclear Energy, Vol. 88, 2016](https://doi.org/10.1016/j.anucene.2015.10.027)
- [Fission yield covariance generation and uncertainty propagation through fission pulse decay heat calculation, Annals of Nuclear Energy, Vol. 69, 2014](https://doi.org/10.1016/j.anucene.2014.01.038)

In [1]:
import sandy
import pandas as pd
from os.path import join
import numpy as np
import random, sys

## Extract FYs and covariance data for U235 fission

In [2]:
za = 92235
tape = sandy.get_endf6_file("jeff_33", "nfpy", za * 10)
fy = sandy.Fy.from_endf6(tape)

In [3]:
energies = fy.data.E.unique()

## Generate perturbation coefficients for all energies and write them to file

Use normalization to chain yields as a constraint to create a covariance matrix. 

In [4]:
tape_rdd = sandy.get_endf6_file("jeff_33", "decay", "all")
rdd = sandy.DecayData.from_endf6(tape_rdd)  # this can take a while

In [5]:
s = rdd.get_chain_yield_sensitivity()
chfy = sandy.fy.get_chain_yields()
set([(row.ZAM, row.E) for i, row in chfy.iterrows()])

{(902270, 'thermal'),
 (902290, 'thermal'),
 (902320, 'fast'),
 (902320, 'high energy'),
 (912310, 'fast'),
 (922320, 'thermal'),
 (922330, 'fast'),
 (922330, 'high energy'),
 (922330, 'thermal'),
 (922340, 'fast'),
 (922340, 'high energy'),
 (922350, 'fast'),
 (922350, 'high energy'),
 (922350, 'thermal'),
 (922360, 'fast'),
 (922360, 'high energy'),
 (922370, 'fast'),
 (922380, 'fast'),
 (922380, 'high energy'),
 (922380, 'spontaneous fission'),
 (932370, 'fast'),
 (932370, 'high energy'),
 (932370, 'thermal'),
 (932380, 'fast'),
 (942380, 'fast'),
 (942390, 'fast'),
 (942390, 'high energy'),
 (942390, 'thermal'),
 (942400, 'fast'),
 (942400, 'high energy'),
 (942400, 'thermal'),
 (942410, 'fast'),
 (942410, 'thermal'),
 (942420, 'fast'),
 (942420, 'high energy'),
 (942420, 'thermal'),
 (952410, 'fast'),
 (952410, 'high energy'),
 (952410, 'thermal'),
 (952421, 'thermal'),
 (952430, 'fast'),
 (962420, 'fast'),
 (962430, 'fast'),
 (962430, 'thermal'),
 (962440, 'fast'),
 (962440, 'spo

Choose among the above fissioning systems for which chain yields are available from [England & Rider, 1993](https://www-nds.iaea.org/endf349/la-ur-94-3106.pdf).

In [6]:
nsmp = 10  # sample size
smp = {}
for e, etext in zip(energies, ["thermal", "fast", "high_energy"]):  # change this for the nuclide you choose
    y_extra = chfy.query(f"E=='{etext}' & ZAM=={za*10}").set_index("A").CHY

    std_extra_info = chfy.query(f"E=='{etext}'").set_index("A").DCHY
    Vy_extra = sandy.CategoryCov.from_stdev(std_extra_info).data

    fy_post, cov_post = fy.gls_update(za*10, e, s, y_extra, Vy_extra)
    
    idx = fy_post.data.query(f"E=={e} & MT==454").index
    ify = fy_post.data.loc[idx]

    seed = random.randrange(2**32 - 1)    # create a seed, need to change it for the different energies
    print(f"sampling IFY for energy {e:.3e} eV...")
    smp[e] = cov_post.sampling(nsmp, seed=seed)   # sample from posterior covariance matrix

sampling IFY for energy 2.530e-02 eV...


sampling IFY for energy 4.000e+05 eV...
sampling IFY for energy 1.400e+07 eV...


In [7]:
with pd.ExcelWriter(f'PERT_{za}_MF8_MT454_COV.xlsx') as writer:
    for e, s in smp.items():
        s.data.to_excel(writer, sheet_name=f'{e:.3e}')

## Read coefficients from perturbation file and generate random FY ENDF-6 files

Skip the part above if you already have the file of perturbations.

In [8]:
smp = pd.read_excel(f'PERT_{za}_MF8_MT454_COV.xlsx', sheet_name=None, index_col=0)

In [9]:
za = 92235
tape = sandy.get_endf6_file("jeff_33", "nfpy", za * 10)
nfpy = sandy.Fy.from_endf6(tape)

In [10]:
smp_min = 0   # write ENDF-6 file only in the sample range [smp_min, smp_max]
smp_max = 9
file_template = "u235_fy_cov_{}.jeff33"
for ismp in range(smp_min, smp_max+1):
    file = file_template.format(ismp)
    f = sandy.Fy(nfpy.data.copy())
    for e, s in smp.items():
        idx_ify = nfpy.data.query(f"E=={float(e)} & MT==454").index
        idx_cfy = nfpy.data.query(f"E=={float(e)} & MT==459").index
        f.data.loc[idx_ify, "DFY"] = f.data.loc[idx_ify, "FY"]            # just for me, i copy the original IFYs where uncertainties should be, so i can compare them to the perturbed ones (anyways I don't use uncertainties)
        f.data.loc[idx_cfy, "DFY"] = f.data.loc[idx_cfy, "FY"]            # same but for CFYs
        f.data.loc[idx_ify, "FY"] *= s[ismp].values                # IMPORTANT, this does not update the CFYs, which in random ENDF-6 file are inconsistent with the perturbed IFYs
        #f = f.apply_qmatrix(922350, e, rdd, keep_fy_index=True)      # Run this if you want to update the CFYs (slower), or else comment it out
    print(f"writing file '{file}'...")
    f.to_endf6(tape).to_file(file)

writing file 'u235_fy_cov_0.jeff33'...


writing file 'u235_fy_cov_1.jeff33'...


writing file 'u235_fy_cov_2.jeff33'...


writing file 'u235_fy_cov_3.jeff33'...


writing file 'u235_fy_cov_4.jeff33'...


writing file 'u235_fy_cov_5.jeff33'...


writing file 'u235_fy_cov_6.jeff33'...


writing file 'u235_fy_cov_7.jeff33'...


writing file 'u235_fy_cov_8.jeff33'...


writing file 'u235_fy_cov_9.jeff33'...
